In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import dill as pickle

import warnings
warnings.filterwarnings("ignore")

In [2]:
DATA_DIR = 'data/'
MODEL_DIR = 'model/'
OUTPUT_DIR = 'output/'

In [3]:
!ls {DATA_DIR}

building_metadata.csv  test.csv   weather_test.csv
sample_submission.csv  train.csv  weather_train.csv


# Load data 

In [4]:
building_metadata = pd.read_csv(DATA_DIR + 'building_metadata.csv')
weather_test = pd.read_csv(DATA_DIR + 'weather_test.csv')
test = pd.read_csv(DATA_DIR + 'test.csv')

In [5]:
map_meter2desc = {0: 'electricity', 1: 'chilledwater', 2: 'steam', 3: 'hotwater'}

In [6]:
numerical = [
    'square_feet', 
    'year_built', 
    'floor_count', 
    'air_temperature', 
    'cloud_coverage', 
    'dew_temperature',
    'precip_depth_1_hr',
    'sea_level_pressure',
    'wind_speed',
]

categorical = [
    'primary_use',
]

# Combine table

In [7]:
df = test
df = df.merge(building_metadata, on='building_id')
df = df.merge(weather_test, on=['site_id', 'timestamp'])
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Inference for each METER

In [9]:
result = pd.DataFrame()

for meter in (0, 1, 2, 3):
    print(map_meter2desc[meter])
    
    df_meter = df.query('meter == {}'.format(meter))
    
    # Load feature preprocessor
    with open(MODEL_DIR + 'feature_preproc_meter{}.p'.format(meter), 'rb') as file:
        feature_preproc = pickle.load(file)
    feature_names = numerical \
        + [ re.sub(r"^(?:x)([0-9])", lambda m: categorical[int(m.group(1))], x) \
             for x in feature_preproc.steps[-1][-1].transformer_list[1][-1].steps[-1][-1].get_feature_names().tolist() ] \
        + ['precip_depth_1_hr_isnan'] \
        + ['wind_direction_sin', 'wind_direction_cos'] \
        + ['month_sin', 'month_cos']
    
    # Load regression model
    with open(MODEL_DIR + 'model_xgboost_meter{}.p'.format(meter), 'rb') as file:
        model = pickle.load(file)
        
    y_pred = model.predict(pd.DataFrame(feature_preproc.transform(df_meter), columns=feature_names))
    result_meter = pd.DataFrame({'row_id': df_meter['row_id'], 'meter_reading': np.round(np.exp(y_pred) - 1, 4)})
    
    # Merge
    result = result.append(result_meter)
    
result = test[['row_id']].merge(result, on='row_id', how='left')

# Default value for missing rows
result['meter_reading'] = result['meter_reading'].fillna(0.0)

electricity
[11:54:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
chilledwater
[11:58:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
steam
[11:59:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
hotwater
[12:00:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


# Save data

In [10]:
result.to_csv(OUTPUT_DIR + 'submit.csv', index=False)